In [1]:
import pandas as pd
import requests
import re
import time

In [2]:
headers = {
    'Referer': 'https://www.rottentomatoes.com/m/notebook/reviews?type=user',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

In [3]:
s = requests.Session()

In [4]:
def get_critic_reviews(url):
    r = requests.get(url)
    movie_id = re.findall(r'(?<=movieId":")(.*)(?=","type)',r.text)[0]

    api_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/criticsReviews/all" #use reviews/userfor user reviews
    
    payload = {
        'direction': 'next',
        'endCursor': '',
        'startCursor': '',
    }
    
    review_data = []
    
    while True:
        r = s.get(api_url, headers=headers, params=payload)
        data = r.json()

        if not data['pageInfo']['hasNextPage']:
            break

        payload['endCursor'] = data['pageInfo']['endCursor']
        payload['startCursor'] = data['pageInfo']['startCursor'] if data['pageInfo'].get('startCursor') else ''

        review_data.extend(data['reviews'])
        time.sleep(1)
    
    return review_data

In [5]:
def get_user_reviews(url):
    r = requests.get(url)
    movie_id = re.findall(r'(?<=movieId":")(.*)(?=","type)',r.text)[0]

    api_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/user" #use reviews/userfor user reviews
    
    payload = {
        'direction': 'next',
        'endCursor': '',
        'startCursor': '',
    }
    
    review_data = []
    
    while True:
        r = s.get(api_url, headers=headers, params=payload)
        data = r.json()

        if not data['pageInfo']['hasNextPage']:
            break

        payload['endCursor'] = data['pageInfo']['endCursor']
        payload['startCursor'] = data['pageInfo']['startCursor'] if data['pageInfo'].get('startCursor') else ''

        review_data.extend(data['reviews'])
        time.sleep(1)
    
    return review_data

In [6]:
#https://www.rottentomatoes.com/m/eternals/reviews
#https://www.rottentomatoes.com/m/black_widow_2021
data_c = get_critic_reviews('https://www.rottentomatoes.com/m/black_widow_2021/reviews')
df_critics = pd.json_normalize(data_c)

In [7]:
data_u = get_user_reviews('https://www.rottentomatoes.com/m/black_widow_2021/reviews')
df_users = pd.json_normalize(data_u)

In [8]:
df_critics.head(3)

,creationDate,isFresh,isRotten,isRtUrl,isTop,reviewUrl,quote,reviewId,scoreOri,scoreSentiment,critic.name,critic.criticPictureUrl,critic.vanity,publication.id,publication.name
0,"Dec 20, 2021",True,False,False,False,https://www.hypable.com/black-widow-review/,Not only do we continue to see Natasha's physi...,2847923,4.5/5,POSITIVE,Karen Rought,http://resizing.flixster.com/q2sGNxdn_BTJb7E4s...,karen-rought,2837,Hypable
1,"Nov 5, 2021",True,False,False,False,https://30flirtyfilm.substack.com/p/issue-51,But the bigger problem with the film is that i...,2837772,,POSITIVE,Cate Young,http://resizing.flixster.com/Gte-H-97idAqD8Fn5...,cate-young,3956,"Thirty, Flirty + Film"
2,"Oct 30, 2021",True,False,False,False,https://www.mychamplainvalley.com/community/mo...,A fabulous director and two awesome stars batt...,2836086,6,POSITIVE,Julia Swift,http://resizing.flixster.com/eFrdpBVieUiCnw4hh...,julia-swift,3340,My Champlain Valley


In [9]:
#df_users.tail(3)

In [10]:
df_users.head(3)

,rating,review,displayName,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId,displayImageUrl
0,3.5,O filme certo na hora errada. A narrativa do f...,Joo Paulo C,False,False,False,False,3.5,5h ago,Joo Paulo C,/user/id/979402311,RT,979402311,NaN
1,4.5,Very Awesome! I enjoyed it a lot. Would see ag...,Jack H,False,False,False,False,4.5,9h ago,Jack H,/user/id/979401921,RT,979401921,NaN
2,2.0,единственный плюс фильма это Красноармеец и водка,,False,False,False,False,2.0,12h ago,,/user/id/978449373,RT,978449373,https://graph.facebook.com/v3.3/19179871983467...


In [11]:
critics_review_text = ' '.join(df_critics['quote'])

In [12]:
user_review_text = ' '.join(df_users['review'])

In [13]:
#critics_review_text

In [14]:
#user_review_text

In [15]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

In [16]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

tokens_critics = []
tokens_critics = word_tokenize(critics_review_text)

In [17]:
print('Number of Critics Tokens =',len(tokens_critics))

Number of Critics Tokens = 12240


In [18]:
tokens_users = []
tokens_users = word_tokenize(user_review_text)

In [19]:
print('Number of Users Tokens =',len(tokens_users))

Number of Users Tokens = 341788


In [20]:
#converts to lower case, and removes punctuation and numbers
wordsFiltered_critics = [tokens.lower() for tokens in tokens_critics if tokens.isalpha()]

In [21]:
print(len(wordsFiltered_critics))

10414


In [22]:
#print(wordsFiltered)

In [23]:
#User Reviews: converts to lower case, and removes punctuation and numbers
wordsFiltered_users = [tokens.lower() for tokens in tokens_users if tokens.isalpha()]

In [24]:
print(len(wordsFiltered_users))

294823


In [25]:
#We initialize the stopwords variable which is a list of words like 
#"The", "I", "and", etc. that don't hold much value as keywords
stop_words = stopwords.words('english')
#print(stop_words)

In [26]:
#some extra stop words are needed after examining the data and word cloud
#these are added
extra_stop_words = ['movies','eternals','movie','verifed', 'tv','rotten', 'tomatoes', 
                    'season', 'verified','review','reviews','view', 'rt','marvel', 'contains','certified',
                    'password','newsletter', 'critic','critics','next','prev','email','film','take','mcu',
                    'one','full','much','feel','feels','thing','things', 'comic','universe','think']
stop_words.extend(extra_stop_words)
#print(stop_words)

In [27]:
# remove stop words from bith tokenised data sets
filtered_words_critics = [word for word in wordsFiltered_critics if word not in stop_words]

In [28]:
# remove stop words from bith tokenised data sets
filtered_words_users = [word for word in wordsFiltered_users if word not in stop_words]

In [29]:
print(len(filtered_words_critics))

5315


In [30]:
#print(filtered_words)

In [31]:
print(len(filtered_words_users))

142738


## Create the Word Cloud

Finally we can create a word cloud backed on the finalised data set of tokenised words. Here we use the WordCloud library to create the word cloud and then the matplotlib library to display the image.

In [32]:
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
mask_upcloud = np.array(Image.open("up_cloud.png"))
mask_downcloud = np.array(Image.open("down_cloud.png"))

In [ ]:
wc_critics = WordCloud(max_words=200, 
               margin=10, 
               background_color='white',
               scale=3,                 # Scaling between computation and drawing
               collocations=True,       # Whether to include collocations (bigrams) of two words. 
               normalize_plurals=True,  # Whether to remove trailing ‘s’ from words. 
               min_word_length=4,       # Minimum number of letters a word must have to include - ignore 2 or 3 letter words.
               relative_scaling = 0.5,
               mask=mask_upcloud,     # change shape of wordcloud into cloud shape
               collocation_threshold=3, # Bigrams must have a Dunning likelihood collocation score greater than this 
                                        # parameter to be counted as bigrams.
               width=700, height=400,
               random_state=1).generate(' '.join(filtered_words_critics))

In [ ]:
wc_users = WordCloud(max_words=200, 
               margin=10, 
               background_color='white',
               scale=3,                 # Scaling between computation and drawing
               collocations=True,       # Whether to include collocations (bigrams) of two words. 
               normalize_plurals=True,  # Whether to remove trailing ‘s’ from words. 
               min_word_length=4,       # Minimum number of letters a word must have to include - ignore 2 or 3 letter words.
               relative_scaling = 0.5,
               mask=mask_downcloud,     # change shape of wordcloud into cloud shape
               collocation_threshold=3, # Bigrams must have a Dunning likelihood collocation score greater than this 
                                        # parameter to be counted as bigrams.
               
               color_func=lambda *args, **kwargs: (255,0,0),
               width=700, height=400,
               random_state=1).generate(' '.join(filtered_words_users))

In [ ]:
#plt.figure(figsize=(20,10))
#plt.imshow(wc)

image_colors = ImageColorGenerator(mask_upcloud)
wc_critics.recolor(color_func=image_colors)

#plt.imshow(wc_critics, interpolation='bilinear')
#plt.axis("off")
#plt.show()
#wc.to_file("/wordcloud.png")

In [ ]:
#plt.figure(figsize=(20,10))
#plt.imshow(wc_users, interpolation='bilinear')
#plt.tight_layout(pad=0)
#plt.axis("off")
#plt.show()

In [ ]:
plt.figure(figsize=(45,20))

plt.subplot(211)
plt.imshow(wc_critics, interpolation='bilinear')
plt.axis("off")

plt.subplot(212)
plt.imshow(wc_users, interpolation='bilinear')
plt.axis("off")

plt.subplots_adjust(hspace=-0.13)
plt.suptitle('Rotten Tomtatoes - Comparison of Critic (Black) to User (Red) Reviews',fontweight ="bold", size=16, x=0.55, y=0.5)
plt.show()

## Word Cloud based on frequency counts

Another alternative when using the WordCloud library is to generate a WordCloud based on the frequency counts. For this you need to build up a table containing two items. The first item is the distinct token and the second column contains the number of times that word/token appears in the text. The following code shows this code and the code to generate the word cloud based on this frequency count.

In [ ]:
from collections import Counter

In [ ]:
# count frequencies
cnt = Counter()
for word in filtered_words:
    cnt[word] += 1

In [ ]:
#print(cnt)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
wc = WordCloud(max_words=1000, margin=10, background_color='white',
               scale=3, 
               relative_scaling = 0.5, 
               collocation_threshold=100,
               width=500, height=400,
               random_state=1).generate_from_frequencies(cnt)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(wc)
plt.axis("off")
plt.show()